In [1]:
import folium
import requests
import xml.etree.ElementTree as ET
import base64
from BaseXClient import BaseXClient
import tkinter as tk
import webbrowser
from folium.plugins import Fullscreen
from folium import plugins
from folium.plugins import MarkerCluster
import os
import webview
from tkhtmlview import HTMLLabel
from tkinter import Tk
from tkinter.filedialog import askopenfilename

from PIL import ImageTk, Image
import PyQt5

from PyQt5 import *
from PyQt5.QtWidgets import *
from PyQt5.QtCore import *
from PyQt5.QtWebEngineWidgets import *
from PyQt5.QtGui import *
from PyQt5.QtCore import *

from folium.plugins import MarkerCluster 

import sys 
import os

In [2]:
# Funciones adicionales (Funcionalidad)
def cargaDBdadoUnMapa(session, mapGiven):

    
    session.execute("create db osm")
    
    try:
        with open(mapGiven, 'rb') as f:
            contenido = f.read()
            session.add("map.osm", contenido.decode())
        # Imprimir un mensaje de confirmación
        print("Mapa OSM cargado en BaseX")
    except Exception as e:
          # Imprimir un mensaje de error si hay un problema
          print("Error al cargar el mapa OSM en BaseX:", e)

def cargaDB2(session):
    
    session.execute("create db osm")
    
    try:
        with open('MT-OSM-master\TAGS\datasets\Madrid\map.osm', 'rb') as f:
            contenido = f.read()
            session.add("map.osm", contenido.decode())
        # Imprimir un mensaje de confirmación
        print("Mapa OSM cargado en BaseX")
    except Exception as e:
          # Imprimir un mensaje de error si hay un problema
          print("Error al cargar el mapa OSM en BaseX:", e)
        
def meterMapa(session):
    # Seleccionar archivo
    raiz = Tk()
    raiz.withdraw()
    mapGiven = askopenfilename()
    
    # Carga el mapa dado el mapa seleccionado
    cargaDBdadoUnMapa(session, mapGiven)
    
def posicionarMapaConCalle(direccion):
    # Convertir la dirección a coordenadas utilizando la API de OSM
    respuesta = requests.get("https://nominatim.openstreetmap.org/search",
                         params={"q": direccion, "format": "json"})
    resultado = respuesta.json()[0] # Tomamos la primera coincidencia
    
    # Mete las coordenadas al mapa
    mapa = folium.Map(location=[resultado["lat"], resultado["lon"]], zoom_start=13)
    
    mapa.save('mapa.html')
    

# Busca los errores en el mapa
def MtGeometry(session, func):
    r = []
    module_namespace = 'mt'
    module_path = 'MT-OSM-master\GEOMETRY\src\mt-osm-geometry.xqm'
    inp = f"import module namespace mt = '{module_namespace}' at '{module_path}';"
    
    for f in func:
        inp += 'mt:' + f + '(.,"",())'
        result = session.query(inp)
        r += result.execute()
        inp = f"import module namespace mt = '{module_namespace}' at '{module_path}';"
    return r

#Función dado un id de NODO, calcula su latitud y longitud
def latandlon(id):
  url = f"https://api.openstreetmap.org/api/0.6/node/{id}"
  response = requests.get(url)
  root = ET.fromstring(response.content)
  node = root.find("node")
  lat = node.attrib["lat"]
  lon = node.attrib["lon"]

  return lat, lon

# Marca los puntos del resultado de la consulta
def marcaPuntos(result):
    mapa = folium.Map(location=[40.4168, -3.7038], zoom_start=15)
    
    #Compacta el string en un xml
    xml_string = '<errores>'
    for r in result:
        xml_string += r
    xml_string += '</errores>'
    
    root = ET.fromstring(xml_string)
    
    #Busca todas las ways
    ways = root.findall('.//way')
    
    #Por cada way, encuentra el primer nodo y lo marca en el mapa
    for w in ways:
        nd_element = w.find('./nd').get('ref')
        lat, lon= latandlon(nd_element)
        
        #Marca un punto en el mapa dado una [latitud, longitud]
        folium.Marker(location=[lat, lon], popup="NoDeadlockTest").add_to(mapa)
    
    mapa.fit_bounds(mapa.get_bounds())
    mapa.save('mapa.html')
    
    return xml_string


def dibujarArea(coordenadas, m):
    i = 1
    for i in range(len(coordenadas)):
        if i == len(coordenadas) - 1:
            folium.PolyLine(locations=[coordenadas[i], coordenadas[0]]).add_to(m)
        folium.PolyLine(locations=[coordenadas[i], coordenadas[i-1]]).add_to(m)
        i+=1

In [3]:
# Funciones adicionales (Interfaz gráfica)
def Dimension():
    #tamaño de la ventana
    pantalla = QApplication.desktop()
    pantallaD = pantalla.screenGeometry(pantalla.primaryScreen())
    anchoP = int(pantallaD.width() * 0.75)
    alturaP = int(pantallaD.height() * 0.75)
    anchoV = int((pantallaD.width() - anchoP) / 2)
    alturaV = int((pantallaD.height() - alturaP) / 2)

    return anchoP, alturaP, anchoV, alturaV
    
def generar_mapa():
    mapa = folium.Map(location=[40.4168, -3.7038], zoom_start=15)
    folium.Marker([40.4168, -3.7038], popup='Madrid').add_to(mapa)
    draw_options = {'polygon': True, 'polyline': False, 'rectangle': True, 'circle': True, 'marker': False, 'circlemarker': False}
    draw_button = plugins.Draw(export=True, draw_options=draw_options)
    draw_button.add_to(mapa)
    mapa.save('mapa.html')

def filtros_seleccionados(checkboxes, mapa):
    session = BaseXClient.Session('localhost', 1984, 'admin', 'admin')
    cargaDB2(session)
    filtros_seleccionados = []
    for checkbox in checkboxes:
        if checkbox.isChecked():
            filtros_seleccionados.append(checkbox.text())
    print(filtros_seleccionados)
    r = MtGeometry(session,filtros_seleccionados)
    marcaPuntos(r)
    session.close()
    

In [4]:
# CLASES INTERFAZ GRÁFICA
class Inicio(QWidget):
    def __init__(self,anchoV,alturaV,anchoP,alturaP):
        super().__init__()
        
        #Creamos Ventana Inicio
        self.setWindowTitle('Ventana con imágenes')
        self.resize(anchoV, alturaV)

        #fondo
        self.fondo = QLabel(self)
        pixmap = QPixmap('img/fondo.png')
        self.fondo.setPixmap(pixmap)
        self.fondo.setScaledContents(True)
        self.fondo.setGeometry(0, 0, anchoP, alturaP+100)
        
        #frente
        self.frente = QLabel(self)
        self.frente.setGeometry(int(anchoP/4), int(alturaP/4), int(anchoP/2), int(alturaP/2))
        self.frente.setStyleSheet("background-color: white; color: #708dc7; border-radius: 10px; font-family:Gilroy; font-size:25px;")
        self.frente.setAlignment(Qt.AlignCenter)  # alineación del contenido
        self.frente.setText("<h1>¡BIENVENIDO!</h1><br><img src='img/logo.png' width='250'>")

        #Cambiamos ventana
        QTimer.singleShot(5000, self.paso_menu)
        
    def paso_menu(self):
        menu = Menu()
        self.parent().setCentralWidget(menu)


class Menu(QWidget):
    def __init__(self):
        super().__init__()
        
        #fondo
        self.fondo = QLabel(self)
        pixmap = QPixmap('img/fondo.png')
        self.fondo.setPixmap(pixmap)
        self.fondo.setScaledContents(True)
        self.fondo.setGeometry(0, 0, anchoP, alturaP+100)
        
        layout = QHBoxLayout(self)

        # Crear botones
        b1 = QPushButton(self)
        b2 = QPushButton(self)
        b3 = QPushButton(self)
        
        b1.setFixedSize(250, 250)
        b2.setFixedSize(250, 250)
        b3.setFixedSize(250, 250)
        
        b1.setStyleSheet("QPushButton { border-image: url('img/posidef.png') 0 0 0 0 stretch stretch; }")
        b2.setStyleSheet("QPushButton { border-image: url('img/elegirposi.png') 0 0 0 0 stretch stretch; }")
        b3.setStyleSheet("QPushButton { border-image: url('img/cargarmap.png') 0 0 0 0 stretch stretch; }")
        
        b1.clicked.connect(self.mapa_def)
        b2.clicked.connect(self.pedir_posi)
        b3.clicked.connect(self.cargar_mapa)
        
        layout.addWidget(b1)
        layout.addWidget(b2)
        layout.addWidget(b3)
  
        self.setLayout(layout)
    
    def mapa_def(self):
        mapaD = MapaD(generar_mapa())
        self.parent().setCentralWidget(mapaD)
        
    def pedir_posi(self):
        pedirP = QDialog(self)
        pedirP.setWindowTitle("Solicitud de Posición")
        pedirP.setGeometry(int(anchoP/2), int(alturaP/2), 200, 150)
        pedirP.setStyleSheet("background-image: url('img/fondo.png');")
        
        texto = QLabel("Introduzca la dirección destino:")
        texto.setStyleSheet("font-family: Arial; font-size: 18px;color: #white;")
        direccion = QTextEdit()
        direccion.setStyleSheet("background-color: white;")
        direccion.setMaximumHeight(100)
        boton = QPushButton("Continuar")
        boton.clicked.connect(lambda: self.enviar_direccion(direccion.toPlainText()))
        
        contenedor = QVBoxLayout()
        contenedor.addWidget(texto)
        contenedor.addSpacerItem(QSpacerItem(0, 10, QSizePolicy.Minimum, QSizePolicy.Expanding))
        contenedor.addWidget(direccion)
        contenedor.addSpacerItem(QSpacerItem(0, 10, QSizePolicy.Minimum, QSizePolicy.Expanding))
        contenedor.addWidget(boton)
        
        pedirP.setLayout(contenedor)
        
        pedirP.exec_()
     
    def enviar_direccion(self, contenido):
        mapaD = MapaD(posicionarMapaConCalle(contenido))
        self.parent().setCentralWidget(mapaD)
        self.close()
        
            
    def cargar_mapa(self):
        session = BaseXClient.Session('localhost', 1984, 'admin', 'admin')
        meterMapa(session)
        mapaD = MapaD(posicionarMapaConCalle("Calle Alcala, Madrid"))
        self.parent().setCentralWidget(mapaD)
        session.close()
        self.close()
    
    
class infoFiltro(QWidget):
    def __init__(self, mensaje):
        super().__init__()

        # Establecer el mensaje
        self.mensaje = mensaje

    def event(self, event):
        if event.type() == QEvent.Enter:
            QToolTip.showText(self.mapToGlobal(self.rect().bottomLeft()), self.mensaje, self)
        elif event.type() == QEvent.Leave:
            QToolTip.hideText()
        return super().event(event)
    
    
class MapaD(QWidget):      
    def __init__(self, funcionMapa):
        super().__init__()
        
        #fondo
        self.fondo = QLabel(self)
        pixmap = QPixmap('img/fondo.png')
        self.fondo.setPixmap(pixmap)
        self.fondo.setScaledContents(True)
        self.fondo.setGeometry(0, 0, anchoP, alturaP+100)
        
        #Creación del mapa
        mapa = QWebEngineView()
        mapa.load(QUrl.fromLocalFile(os.path.realpath("mapa.html")))
        mapa.setFixedSize(int(anchoP*4/5),alturaP)
        mapaL = QVBoxLayout()
        mapaL.addWidget(mapa)
         
        #Filtros
        lista = QGridLayout()
        lista.setSpacing(10)
        elementos = ['NoDeadlockTest', 'NoIsolatedWayTest', 'ExitWayTest', 'EntranceWayTest', 'ExitRAboutTest', 'EntRAbaoutTest', 'ConnectedTest', 'AreaNoIntTest', 'NoOverlapTest']
        infosF = ['Al seleccionar este filtro, buscaremos\naquellos puntos que presentan direcciones\nopuestas y no existe una salida',
                  'Al seleccionar este filtro, buscaremos\naquellas vías que tienen entradas y/o salidas',
                  'Al seleccionar este filtro, buscaremos\naquellas vias que tienen al menos una salida',
                  'Al seleccionar este filtro, buscaremos\naquellas vias que tienen al menos una entrada',
                  'Al seleccionar este filtro, buscaremos\nrotondas que no tengan salidas',
                  'Al seleccionar este filtro, buscaremos\nrotondas que no tengan entradas',
                  'Al seleccionar este filtro, buscaremos\nque existan al menos una forma conectada a la etiqueta',
                  'Al seleccionar este filtro, buscaremos\naquellas áreas que no están intersectadas',
                  'Al seleccioanr este filtro, buscaremos\naquellas zonas que no se encuentran superpuestas por otras']
        checkboxes = []
        for i, elemento in enumerate(elementos):
            checkbox = QCheckBox(elemento)
            info = infosF[i] if i < len(infosF) else ''
            infoF = infoFiltro(info)
            lista.addWidget(checkbox, i, 0)
            lista.addWidget(infoF, i, 1)
            checkboxes.append(checkbox)
            

        #Contendor Filtros
        texto = QLabel("Seleccione los filtros\n que desea aplicar:")
        texto.setStyleSheet("font-family: Arial; font-size: 18px;color: #white;")
        botonL = QHBoxLayout()
        b1 = QPushButton('Ejecutar')
        b2 = QPushButton('Selecionar Zona')
        b1.clicked.connect(lambda:filtros_seleccionados(checkboxes, mapa))
        b1.clicked.connect(self.refresh_map)
        botonL.addStretch()
        botonL.addWidget(b1)
        botonL.addWidget(b2)

        filtroL = QVBoxLayout()
        filtroL.addWidget(texto)
        filtroL.addLayout(lista)
        filtroL.addSpacerItem(QSpacerItem(0, 35, QSizePolicy.Minimum, QSizePolicy.Expanding))
        filtroL.addLayout(botonL)

        #Contenedor General
        mapaW = QWidget()
        mapaW.setLayout(mapaL)

        filtroW = QWidget()
        filtroW.setLayout(filtroL)

        main = QHBoxLayout()
        main.addWidget(mapaW,4)
        main.addWidget(filtroW,1)

        self.setLayout(main)
        
    def refresh_map(self):
        self.mapa.reload()

In [ ]:
# MAIN
if __name__ == '__main__':
    app = QApplication(sys.argv)
    anchoP, alturaP, anchoV, alturaV = Dimension()
    
    inicio = Inicio(anchoV,alturaV,anchoP,alturaP)
    
    ventanaP = QMainWindow()
    ventanaP.setCentralWidget(inicio)
    
    ventanaP.setWindowTitle("MapFlaws")
    ventanaP.setGeometry(anchoV,alturaV,anchoP,alturaP)
    ventanaP.setWindowIcon(QIcon('img/logo.ico'))
    ventanaP.show()
    
    sys.exit(app.exec_())

Mapa OSM cargado en BaseX
['NoDeadlockTest']


TypeError: MapaD.refresh_map() takes 0 positional arguments but 2 were given